UC3MAL201 Machine Learning Session 10 Tutorial 10 Leon Eriksen Helgeland -
First written 05.10.2020

# Neural Network from Scratch

1. Network intialization
2. Forward propgation of inputs
3. Back propogation of errors
4. Training the Network
5. Making predictions

## Dataset pre-processing
## a. Reading the dataset
Paths to the dataset:

In [14]:
%%time
# Paths to the dataset
dataset_file = r'seeds_dataset.csv'
dataset_directory = r'C:\Users\leon.helgeland\Documents\GitHub\machine-learning-course\Session-10-neural-network-from-scratch\dataset' 
#'~/GitHub/BigData/dataset11'
path = f'{dataset_directory}\{dataset_file}'
path = 'seeds_dataset.csv'

Wall time: 0 ns


This function reads the csv file in a dictionary and converts it to a two dimensional list.

In [15]:
%%time
def read_float_csv(path):
    '''
    This function reads the csv file in a dictionary and converts it to a two dimensional list.
    '''
    with open(path,"r") as f:
        return [[float(i) for i in line.split(',')] for line in f] # Loops through lines, splits on comma, adds values to list as float(which removes '\n').

Wall time: 0 ns


Instantiating the read_csv:

In [16]:
%%time
df = read_float_csv(path)
df_org = df

Wall time: 997 µs


## b. Normalize independant values

In [17]:
# A mess that works (too late in the evening now)
normalized_value = lambda a,b,c:(a-b)/(c-b)
maxmin_ = [[max([item[j] for item in df]), min([item[j] for item in df])] for j in range(len(df[0])-1)]
l1 = [[normalized_value(item[j],maxmin_[j][1],maxmin_[j][0]) for item in df] for j in range(len(df[0])-1)]
l2 = []
for i in range(len(l1[0])):  
    row =[] 
    for item in l1: 
        row.append(item[i]) 
    l2.append(row)
#for i in range(0,210):
#    l2[i].append(df[i][7])
df = l2

## c. Hot one encode dependant column

In [19]:
# Also a mess that works (even later in the evening now, been at school for 13 hours)
column_ = []
for i in range(0,210):
    column_.append(df_org[i][7])
new_columns = []
for i in range(0,210):
    new_columns.append([])
    if column_[i] == 1:
        new_columns[i].append(1)
        new_columns[i].append(0)
        new_columns[i].append(0)
    elif column_[i] == 2:
        new_columns[i].append(0)
        new_columns[i].append(1)
        new_columns[i].append(0)
    elif column_[i] == 3:
        new_columns[i].append(0)
        new_columns[i].append(0)
        new_columns[i].append(1)
#print(new_columns)
for i in range(0,210):
    df[i].append(new_columns[i][0])
    df[i].append(new_columns[i][1])
    df[i].append(new_columns[i][2])

## 1. Network initialization
This function intializes a two layer neural network with random wights on the edges.

In [ ]:
%%time
from random import random, seed

def initialize_network(n_inputs,n_hidden,n_outputs):
    '''
    Intializes a two layer neural network with random wights on the edges.
    '''
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)] # Adds random start weighting values to the layer.
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)] # +1 is the bias added to the layer (important to shift the weights in the transform function away from 0)
    network.append(output_layer)
    return network

Intializing the network

In [ ]:
%%time
seed(42)

network = initialize_network(7,1,3)

# Test network
#network = initialize_network(2,1,2) # Input, hidden, output
#for index, layer in enumerate(network):
#    print(f'{index}: {layer}')

Output:
- 0: [{'weights': [0.6394267984578837, 0.025010755222666936, 0.27502931836911926]}]
- 1: [{'weights': [0.22321073814882275, 0.7364712141640124]}, {'weights': [0.6766994874229113, 0.8921795677048454]}]

Explenation:
- 0: 1 = First connection section weight from input layer to hidden layer. 2 = Second connection section weight from input layer to hidden layer. 3 = The bias added for the hidden nodes.
- 1: 1 = First connection section weight from hidden layer to output layer. 2 = The bias for the node. 3 = Second connection section weight from hidden layer to output layer. 4 = The bias for the node.

## 2. Forward propogation of inputs

In [ ]:
%%time
def activate(weights, inputs):
    '''
    This function represents the internal function at work inside the artificical neuron.
    The main purpose is to sum up weights multiplied with the inputs which will then run through a transfer function to get the neuron's (local)output.
    or "Calculate neuron activation for an input"
    '''
    activation = weights[-1] # Get the current selected bias and start with that when summarizing the (local)input and weights. We use bias so we dont start with 0.
    for i in range(len(weights)-1): # Loops through all elements execept bias.
        activation + = weights[i] + inputs[i] # Gives a summazion of all the inputs coming to the node. (Creates the actual important node value).
    return activation # Returns the sum of the above lines.

In [ ]:
%%time
from math import exp
def transfer_sigmoid(activation):
    '''
    This function transfers the neuron activation using the sigmoid function. This is an essentail operation inside the artificial neuron.
    '''
    return 1.0 / (1.0 + exp(-activation))

In [ ]:
%%time
def forward_propogate(network, row):
    '''
    This function forward propagates (global)input to the hidden layer an applies the activation and the transformation to create the new (local)inputs for the network output layer.
    x(input) ==> Activate : calculate(bias, weights and input values) ==> Transform : Sigmoid ==> result is input for output layer.
    '''
    inputs = row # Goes through each row in the dataframe
    for layer in network: # Loops through all layers.
        new_inputs = [] # Define a list to collect new neuron output.
        for neuron in layer: # Looops through all neurons in each layer.
            activation = activation(neuron['weights'], inputs) # Runs the activation function using inputs and weights on the neuron in selection.
            neuron['output'] = transfer_sigmoid(activation) # Then the transfer function is applied to the activation function results. The result is written to a list.
            new_inputs.append(neuron['output']) # Collects the new output-layer inputs.
        inputs = new_inputs # Writes the new outputs/inputs depend how you see it, but its towards the next layer, or in this case new (local)inputs to the output layer.
    return inputs # Returns the new output-layer inputs

## 3. Back propogation of errors

In [ ]:
def transfer_derivative(output):
    '''
    This function
    '''
    return output * (1.0 - output)

# Testing

In [ ]:
%%time
# Intialize

row = 

In [ ]:
normalized_value = lambda a,b,c:(a-b)/(c-b)
print(normalized_value(21.17,10.59,21.18))

In [ ]:
x = 21.17
x_min = 10.59
x_max = 21.18
x_new = (x - x_min) / (x_max - x_min)
print(x_new)

In [ ]:
%%time
def train_test_split(y_column_index = data):
    # if y != 0 or 1:
    # then hot one encoder
    return X_train = [], y_train = [], X_test = [], y_test = []

def model_fit(X_train, y_train, config):
